In [1]:
import csv
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import pickle

In [2]:
# 定义年份和季度范围
years = range(2019, 2023)
quarters = ['q1', 'q2', 'q3', 'q4']

Bike_trips = pd.DataFrame()
# 遍历年份和季度
for year in years:
    if year == 2023:
        quarters = ['q1']
        # 构建文件名
        file_name = f"../data/PA/share_bike_CBG/indego-trips-{year}-{quarter}_CBG.csv"

        # 读取CSV文件
        data = pd.read_csv(file_name)
        
        Bike_trips = Bike_trips.append(data, ignore_index = True)
        break
    
    for quarter in quarters:
        # 构建文件名
        file_name = f"../data/PA/share_bike_CBG/indego-trips-{year}-{quarter}_CBG.csv"

        # 读取CSV文件
        data = pd.read_csv(file_name)
        
        Bike_trips = Bike_trips.append(data, ignore_index = True)      

/var/folders/tw/gz2w91cj3fj4x6fqjsqpt1m80000gn/T/ipykernel_9342/1628518722.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bike_trips = Bike_trips.append(data, ignore_index = True)
/var/folders/tw/gz2w91cj3fj4x6fqjsqpt1m80000gn/T/ipykernel_9342/1628518722.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bike_trips = Bike_trips.append(data, ignore_index = True)
/var/folders/tw/gz2w91cj3fj4x6fqjsqpt1m80000gn/T/ipykernel_9342/1628518722.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Bike_trips = Bike_trips.append(data, ignore_index = True)
/var/folders/tw/gz2w91cj3fj4x6fqjsqpt1m80000gn/T/ipykernel_9342/1628518722.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

In [37]:
Bike_trips

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Start_OBJECTID,End_OBJECTID
0,306773863,8,2019-01-01 00:19:00,2019-01-01 00:27:00,3049,39.945091,-75.142502,3007,39.945171,-75.159927,14495,30.0,One Way,Indego30,standard,124.0,582.0
1,306773862,7,2019-01-01 00:30:00,2019-01-01 00:37:00,3005,39.947330,-75.144028,3007,39.945171,-75.159927,5332,1.0,One Way,Day Pass,standard,576.0,582.0
2,306773861,13,2019-01-01 00:52:00,2019-01-01 01:05:00,3166,39.972351,-75.134453,3169,39.953819,-75.142632,14623,30.0,One Way,Indego30,standard,235.0,123.0
3,306773860,9,2019-01-01 00:55:00,2019-01-01 01:04:00,3058,39.967159,-75.170013,3103,39.977139,-75.179398,11706,30.0,One Way,Indego30,standard,157.0,1260.0
4,306773859,12,2019-01-01 01:05:00,2019-01-01 01:17:00,3182,39.950809,-75.169533,3028,39.940609,-75.149582,11039,30.0,One Way,Indego30,standard,198.0,761.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3344335,579499101,23,12/31/2022 23:44,1/1/2023 0:07,3244,39.938648,-75.166740,3287,39.943668,-75.216057,14674,30.0,One Way,Indego30,standard,941.0,1034.0
3344336,579499099,17,12/31/2022 23:45,1/1/2023 0:02,3295,39.950279,-75.160271,3054,39.962502,-75.174202,23113,30.0,One Way,Indego30,electric,310.0,150.0
3344337,579499097,8,12/31/2022 23:51,12/31/2022 23:59,3255,39.950951,-75.164383,3046,39.950119,-75.144722,16897,30.0,One Way,Indego30,electric,132.0,575.0
3344338,579499095,23,12/31/2022 23:54,1/1/2023 0:16,3255,39.950951,-75.164383,3113,39.974720,-75.197807,22646,30.0,One Way,Indego30,electric,132.0,200.0


In [3]:
# drop na rows where 'Start_OBJECTID' or 'End_OBJECTID' is na
Bike_trips = Bike_trips.dropna(subset=['Start_OBJECTID', 'End_OBJECTID'])

In [4]:
# Define a bike_demand (x,y,z), x is the Start_OBJECTID, y is the End_OBJECTID
# order X_index and Y_index by OBJECTID
X_index = Bike_trips['Start_OBJECTID'].unique()
Y_index = Bike_trips['End_OBJECTID'].unique()
bike_demand = [[[0] for _ in range(len(X_index))] for _ in range(len(Y_index))]

In [40]:
from datetime import datetime

def hours_to_timeline(dt, timeline = datetime(2019, 1, 1, 0, 0, 0)):
    diff = dt - timeline
    hours = diff.total_seconds() / 3600
    return int(hours)

timeline = datetime(2019, 1, 1, 0, 0, 0)
dt = datetime(2023, 12, 31, 23, 56, 0)
# test
print(hours_to_timeline(dt, timeline)) 

def four_hours_to_timeline(dt, timeline = datetime(2019, 1, 1, 0, 0, 0)):
    diff = dt - timeline
    hours = diff.total_seconds() / 3600
    return int(hours / 4)

print(four_hours_to_timeline(dt, timeline))

43823
10955


In [91]:
bike_demand = {}

for index, trip in Bike_trips.iterrows():
    start_id = int(trip['Start_OBJECTID'])
    end_id = int(trip['End_OBJECTID'])
    try:
        start_time = datetime.strptime(trip['start_time'], "%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(trip['end_time'], "%Y-%m-%d %H:%M:%S")
    except:
        start_time = datetime.strptime(trip['start_time'], "%m/%d/%Y %H:%M")
        end_time = datetime.strptime(trip['end_time'], "%m/%d/%Y %H:%M")

    # Get the time step
    start_time_step = hours_to_timeline(start_time, timeline)
    end_time_step = hours_to_timeline(end_time, timeline)

    for hour in range(start_time_step, end_time_step+1):
        key = (start_id, end_id, hour)
        if key in bike_demand:
            bike_demand[key] += 1
        else:
            bike_demand[key] = 1

In [99]:

with open("../data/PA/bike_demand_dict.pkl", "wb") as f:
    pickle.dump(bike_demand, f)

In [3]:
# load '../data/PA/bike_demand_dict.pkl'
bike_demand = pickle.load(open('../data/PA/bike_demand_dict.pkl', 'rb'))

In [4]:
bike_demand

{(124, 582, 0): 1,
 (576, 582, 0): 1,
 (235, 123, 0): 1,
 (235, 123, 1): 1,
 (157, 1260, 0): 1,
 (157, 1260, 1): 1,
 (198, 761, 1): 1,
 (575, 154, 1): 1,
 (315, 404, 1): 1,
 (168, 1318, 1): 1,
 (1017, 582, 1): 1,
 (125, 309, 1): 1,
 (153, 378, 1): 1,
 (153, 378, 2): 1,
 (576, 609, 1): 1,
 (131, 1011, 1): 1,
 (131, 1011, 2): 1,
 (577, 761, 1): 1,
 (577, 761, 2): 1,
 (1324, 1179, 1): 1,
 (1324, 1179, 2): 1,
 (378, 582, 1): 1,
 (378, 582, 2): 1,
 (324, 576, 2): 1,
 (258, 157, 2): 2,
 (761, 934, 2): 1,
 (1250, 378, 2): 1,
 (130, 944, 2): 1,
 (131, 579, 2): 1,
 (131, 579, 3): 1,
 (131, 579, 4): 1,
 (131, 579, 5): 1,
 (131, 579, 6): 1,
 (131, 579, 7): 1,
 (131, 579, 8): 1,
 (131, 579, 9): 1,
 (404, 414, 2): 1,
 (1333, 676, 2): 1,
 (1196, 206, 2): 1,
 (1196, 206, 3): 1,
 (123, 761, 2): 1,
 (123, 761, 3): 1,
 (132, 1200, 2): 1,
 (132, 1200, 3): 1,
 (761, 199, 2): 1,
 (761, 199, 3): 1,
 (1197, 1202, 3): 1,
 (1032, 258, 3): 1,
 (1032, 258, 4): 1,
 (515, 203, 3): 1,
 (235, 235, 3): 1,
 (582, 1014

In [6]:
filename = "../data/train/adjacent_id.txt"

with open(filename, "r") as file:
    adjacent_id = [int(line.strip()) for line in file]

print(adjacent_id)

[10, 53, 67, 122, 123, 124, 125, 126, 129, 130, 131, 132, 150, 152, 153, 154, 156, 157, 168, 197, 198, 199, 200, 201, 203, 206, 208, 210, 215, 216, 235, 236, 245, 246, 253, 255, 256, 258, 276, 282, 289, 309, 310, 311, 312, 314, 315, 324, 326, 334, 348, 378, 403, 404, 405, 406, 414, 420, 495, 503, 513, 515, 575, 576, 577, 578, 579, 580, 581, 582, 583, 609, 665, 674, 675, 676, 688, 690, 694, 696, 701, 761, 779, 780, 781, 933, 934, 937, 939, 941, 943, 944, 955, 1011, 1013, 1014, 1015, 1017, 1020, 1029, 1031, 1032, 1038, 1043, 1095, 1103, 1104, 1105, 1125, 1154, 1158, 1177, 1179, 1191, 1193, 1194, 1196, 1197, 1200, 1202, 1204, 1247, 1250, 1252, 1256, 1257, 1259, 1260, 1284, 1318, 1320, 1324, 1326, 1327, 1333, 1334]


In [11]:
adjacent_id.index(53)

1

In [12]:
bike_demand = np.zeros((136, 136, 43823))
region_ids = adjacent_id
for index, trip in Bike_trips.iterrows():
    start_id = int(trip['Start_OBJECTID'])
    end_id = int(trip['End_OBJECTID'])
    if not (start_id in region_ids and end_id in region_ids):
        continue
    try:
        start_time = datetime.strptime(trip['start_time'], "%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(trip['end_time'], "%Y-%m-%d %H:%M:%S")
    except:
        start_time = datetime.strptime(trip['start_time'], "%m/%d/%Y %H:%M")
        end_time = datetime.strptime(trip['end_time'], "%m/%d/%Y %H:%M")
    # Get the index
    start_index = region_ids.index(start_id)
    end_index = region_ids.index(end_id)
    # Get the time step
    start_time_step = hours_to_timeline(start_time, timeline)
    end_time_step = hours_to_timeline(end_time, timeline)

    for hour in range(start_time_step, end_time_step+1):
        bike_demand[start_index, end_index, hour] += 1

In [10]:
bike_demand = np.zeros((136, 1, 43823))
region_ids = adjacent_id
for index, trip in Bike_trips.iterrows():
    start_id = int(trip['Start_OBJECTID'])
    if not (start_id in region_ids):
        continue
    try:
        start_time = datetime.strptime(trip['start_time'], "%Y-%m-%d %H:%M:%S")
    except:
        start_time = datetime.strptime(trip['start_time'], "%m/%d/%Y %H:%M")
    # Get the index
    start_index = region_ids.index(start_id)
    # Get the time step
    start_time_step = hours_to_timeline(start_time, timeline)
    bike_demand[start_index, 0, start_time_step] += 1

In [45]:
embedding = np.load('../data/train/embeddings.npy')
embedding.shape

(136, 1536)

In [47]:
bike_demand_em = np.zeros((136, 1537, 43823))
for time_step in range(43823):
    bike_demand_em[:, 1:, time_step] = embedding
region_ids = adjacent_id
for index, trip in Bike_trips.iterrows():
    start_id = int(trip['Start_OBJECTID'])
    if not (start_id in region_ids):
        continue
    try:
        start_time = datetime.strptime(trip['start_time'], "%Y-%m-%d %H:%M:%S")
    except:
        start_time = datetime.strptime(trip['start_time'], "%m/%d/%Y %H:%M")
    # Get the index
    start_index = region_ids.index(start_id)
    # Get the time step
    start_time_step = hours_to_timeline(start_time, timeline)
    bike_demand_em[start_index, 0, start_time_step] += 1

KeyboardInterrupt: 

In [20]:
len(Bike_trips)

3289920

In [43]:
bike_demand[:,0,20121]

array([ 0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  4., 10.,
        2.,  0.,  1.,  0.,  2.,  0.,  3.,  1.,  0.,  2.,  0.,  1.,  0.,
        0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  2.,  1.,  2.,  1.,  1.,  0.,  2.,  0.,  0.,  1.,
        0.,  2.,  3.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  2.,  0.,  0.,
        0.,  2.,  1.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  1.,  2.,  0.,  1.,
        0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  3.,
        3.,  0.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.])

In [11]:
np.save('../data/train/X.npy', bike_demand)

In [3]:
x = np.load('../data/train/X.npy')
x.shape

(136, 1, 43823)

In [23]:
adjacent_id.index(124)

5

In [28]:
# split into train and test sets
split_index = int(bike_demand.shape[2] * 0.7)
train_input, test_input = bike_demand[:, :, :split_index], bike_demand[:, :, split_index:]
np.save('../data/train/trian_input.npy', train_input)
np.save('../data/train/test_input.npy', test_input)

In [24]:
train_input.shape

(136, 136, 30676)